# Entregable semana 7
## Sobre la base del entregable de la semana 5
#### - Se agrego un DF que incorpora datos del top 50 actuales del ranking FIFA con el objetivo de saber como les fue durante la fase de grupos del mundial.
#### - Se tomo en cuenta que al momento de levantar los datos de la web, se usaba la posición de los DF siendo esto suceptible a fallos (segun lo indicado).Para resolverlo se usaron las sentencias "try","except" convinado con " if(df.columns==**)"




In [1]:
!pip install pandas

In [2]:
!pip install lxml

In [3]:
!pip install python-dotenv

In [10]:
# para conectarse con cluster de AWS
!pip install redshift_connector 

In [2]:
import pandas as pd
import pickle


In [5]:
# (Creando la conexion)
import os
import dotenv

In [31]:
info = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2022')

In [7]:
# cargamos las variables de entorno
dotenv.load_dotenv()  

True

In [8]:
# leemos las varaibles de entorno
usuario = os.getenv('User')
passwd = os.getenv('Password')
host = os.getenv('host')
database = os.getenv('db')

In [11]:
import redshift_connector

conn = redshift_connector.connect(
     host=host,
     database=database,
     port=5439,
     user=usuario,
     password=passwd
  )

InterfaceError: ('communication error', PermissionError(13, 'Intento de acceso a un socket no permitido por sus permisos de acceso', None, 10013, None))

In [11]:
my_schema='federicobergada_coderhouse'

conn.rollback() #regresa al inicio del programa, antes de la ejecución de estos cambios. De esta manera, no se deja ninguna operación a medias.
conn.autocommit = True
cursor = conn.cursor()
## Aca hay que cambiar la tabla
cursor.execute(f'CREATE TABLE if not exists {my_schema}.faseDeGruposMundial(Selección VARCHAR(30),Pts INT, PJ INT, PG INT, PE INT, PP INT, GF INT, GC INT, Dif VARCHAR(5))')


In [43]:
# Aca habría que ver la forma de cambia la forma de subir la tabla  a redsfhit
#Este codigo hay que remplazarlo con el que esta abajo
# Insert Dataframe into SQL Server:
import numpy as np

sql = "INSERT INTO federicobergada_coderhouse.faseDeGruposMundial(Selección,Pts,PJ,PG,PE,PP,GF,GC,Dif) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)" 
for k in range(8,64,7):
    for i in range(len(info[k].index)):
        list_item = []
        for j in range(9):
            if type(list(info[k].iloc[i])[j])  is np.int64:        
                list_item.append(list(info[k].iloc[i])[j].item())
            else:
                list_item.append(list(info[k].iloc[i])[j])
             #valores = list(info[k].iloc[i])
        #print(type(list_item[len(list_item)-1]))
        cursor.execute(sql, list_item)
conn.commit() 
cursor.close()



NameError: name 'cursor' is not defined

In [32]:
# Lee los datos de la página, filtranndo por aquellos df con columna dif
dfs = pd.read_html('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2022',match='Dif')      

In [42]:
# con el try y el except filtra solo los df que tienen las columnas mencionadas (son los df de los grupos) y nos independizamos de llamar a los dataframes por su ubicación 
#con el resto del codigo permite insertar fila por fila
import numpy as np

for df in dfs: 
    try:
        if (df.columns == ['Selección', 'Pts', 'PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'Dif']).all():
            for i in range(len(df.index)):
                list_item = []
                for j in range(9):
                    if type(list(df.iloc[i])[j])  is np.int64:
                        list_item.append(list(df.iloc[i])[j].item())
                    else:
                        list_item.append(list(df.iloc[i])[j])
                print(list_item)                       
    except ValueError:
        pass

['Países Bajos', 7, 3, 2, 1, 0, 5, 1, '4']
['Senegal', 6, 3, 2, 0, 1, 5, 4, '1']
['Ecuador', 4, 3, 1, 1, 1, 4, 3, '1']
['Catar', 0, 3, 0, 0, 3, 1, 7, '–6']
['Inglaterra', 7, 3, 2, 1, 0, 9, 2, '7']
['Estados Unidos', 5, 3, 1, 2, 0, 2, 1, '1']
['Irán', 3, 3, 1, 0, 2, 4, 7, '–3']
['Gales', 1, 3, 0, 1, 2, 1, 6, '–5']
['Argentina', 6, 3, 2, 0, 1, 5, 2, '3']
['Polonia', 4, 3, 1, 1, 1, 2, 2, '0']
['México', 4, 3, 1, 1, 1, 2, 3, '–1']
['Arabia Saudita', 3, 3, 1, 0, 2, 3, 5, '–2']
['Francia', 6, 3, 2, 0, 1, 6, 3, '3']
['Australia', 6, 3, 2, 0, 1, 3, 4, '–1']
['Túnez', 4, 3, 1, 1, 1, 1, 1, '0']
['Dinamarca', 1, 3, 0, 1, 2, 1, 3, '–2']
['Japón', 6, 3, 2, 0, 1, 4, 3, '1']
['España', 4, 3, 1, 1, 1, 9, 3, '6']
['Alemania', 4, 3, 1, 1, 1, 6, 5, '1']
['Costa Rica', 3, 3, 1, 0, 2, 3, 11, '–8']
['Marruecos', 7, 3, 2, 1, 0, 4, 1, '3']
['Croacia', 5, 3, 1, 2, 0, 4, 1, '3']
['Bélgica', 4, 3, 1, 1, 1, 1, 2, '–1']
['Canadá', 0, 3, 0, 0, 3, 2, 7, '–5']
['Brasil', 6, 3, 2, 0, 1, 3, 1, '2']
['Suiza', 6, 3, 2, 0

In [29]:
# Me traigo el top 50 del ranking Fifa
url='https://es.wikipedia.org/wiki/Anexo:Estad%C3%ADsticas_de_la_clasificaci%C3%B3n_mundial_de_la_FIFA'
fifadfs=pd.read_html(url,na_values='$Nulls$')

for fifadf in fifadfs: 
    try:
        if (fifadf.columns == ['Pos.', 'V', 'Selección', 'Continente', 'Confederación','Total puntos']).all():
            print(fifadf)
            break
    except ValueError:
        pass

    Pos.    V        Selección         Continente Confederación  Total puntos
0      1  1.0        Argentina    América del Sur      Conmebol       1840.93
1      2  1.0          Francia             Europa          UEFA       1838.45
2      3  2.0           Brasil    América del Sur      Conmebol       1834.21
3      4  NaN          Bélgica             Europa          UEFA       1792.53
4      5  NaN       Inglaterra             Europa          UEFA       1792.43
5      6  NaN     Países Bajos             Europa          UEFA       1731.23
6      7  NaN          Croacia             Europa          UEFA       1730.02
7      8  NaN           Italia             Europa          UEFA       1713.66
8      9  NaN         Portugal             Europa          UEFA       1707.22
9     10  NaN           España             Europa          UEFA       1682.85
10    11  NaN        Marruecos             África           CAF       1677.79
11    12  NaN            Suiza             Europa          UEFA 

In [58]:
dffg=pd.DataFrame()
import numpy as np
for df in dfs: 
    try:
        if (df.columns == ['Selección', 'Pts', 'PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'Dif']).all():
            for i in range(len(df.index)):
                list_item = []
                for j in range(9):
                    if type(list(df.iloc[i])[j])  is np.int64:
                        list_item.append(list(df.iloc[i])[j].item())
                    else:
                        list_item.append(list(df.iloc[i])[j])
                dffg = pd.concat([dffg, pd.DataFrame([list_item])], ignore_index=True)
                     
    except ValueError:
        pass
dffg.rename(columns = {dffg.columns[0]:'Selección',
                                    dffg.columns[1]:'Pts',
                                    dffg.columns[2]:'PJ',
                                    dffg.columns[3]:'PG',
                                    dffg.columns[4]:'PE',
                                    dffg.columns[5]:'PP',
                                    dffg.columns[6]:'GF',
                                    dffg.columns[7]:'GC',
                                    dffg.columns[8]:'Dif'},inplace = True)
dffg

,Selección,Pts,PJ,PG,PE,PP,GF,GC,Dif
0,Países Bajos,7,3,2,1,0,5,1,4
1,Senegal,6,3,2,0,1,5,4,1
2,Ecuador,4,3,1,1,1,4,3,1
3,Catar,0,3,0,0,3,1,7,–6
4,Inglaterra,7,3,2,1,0,9,2,7
5,Estados Unidos,5,3,1,2,0,2,1,1
6,Irán,3,3,1,0,2,4,7,–3
7,Gales,1,3,0,1,2,1,6,–5
8,Argentina,6,3,2,0,1,5,2,3
9,Polonia,4,3,1,1,1,2,2,0


In [70]:
# la idea es ver como les fue a los top 50 (actuales en el último mundial)
# merge entre las dos tablas, y se eliminan los que son nulos, es decir quienes no estaban en el top 50 y lo ordeno por posición
dffg_fifa_t = pd.merge(dffg, fifadf, on='Selección',how='left')
dffg_fifa = dffg_fifa_t[['Selección', 'Pts', 'PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'Dif','Pos.', 'Continente', 'Confederación','Total puntos']].dropna().sort_values(by=['Pos.'],ignore_index= True)
dffg_fifa.dtypes

Selección         object
Pts                int64
PJ                 int64
PG                 int64
PE                 int64
PP                 int64
GF                 int64
GC                 int64
Dif               object
Pos.             float64
Continente        object
Confederación     object
Total puntos     float64
dtype: object